(TensorShape([60000, 28, 28]), TensorShape([60000]))

In [213]:
import tensorflow as tf
from numpy.random import sample
from tensorflow import keras, losses
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

(X,y),_ = datasets.mnist.load_data()
X = tf.convert_to_tensor(X,dtype=tf.float32) / 255.
y = tf.convert_to_tensor(y,dtype=tf.int32)
X.shape, y.shape
train_db = tf.data.Dataset.from_tensor_slices((X, y)).batch(128)
w1 = tf.Variable(tf.random.truncated_normal(shape=(784,256), dtype=tf.float32, stddev=0.1))
b1 = tf.Variable(tf.zeros(256))
w2 = tf.Variable(tf.random.truncated_normal(shape=(256,128), dtype=tf.float32, stddev=0.1))
b2 = tf.Variable(tf.zeros(128))
w3 = tf.Variable(tf.random.truncated_normal(shape=(128,10), dtype=tf.float32, stddev=0.1))
b3 = tf.Variable(tf.zeros(10))
lr = 0.001
def model():
    for step,(x,y) in enumerate(train_db):
        """
        x [128,28,28]
        y [128]
        """
        x = tf.reshape(x, (-1,784))


        with tf.GradientTape() as tape:
            h1 = x @ w1 + b1
            h1 = tf.nn.relu(h1)
            h2 = h1 @ w2 + b2
            h2 = tf.nn.relu(h2)
            out = h2 @ w3 + b3

            y_one_hot = tf.one_hot(y, depth=10)

            loss = tf.square(y_one_hot - out)
            loss = tf.reduce_mean(loss)

        grads = tape.gradient(loss, [w1, b1, w2, b2, w3, b3])
        w1.assign_sub(lr * grads[0])
        b1.assign_sub(lr * grads[1])
        w2.assign_sub(lr * grads[2])
        b2.assign_sub(lr * grads[3])
        w3.assign_sub(lr * grads[4])
        b3.assign_sub(lr * grads[5])
        if step % 100 == 0:
            print("loss",step,float(loss))
for i in range(10):
    print(i)
    model()


0
loss 0 0.47273269295692444
loss 100 0.22516962885856628
loss 200 0.20336151123046875
loss 300 0.1925262212753296
loss 400 0.19787970185279846
1
loss 0 0.18383356928825378
loss 100 0.16703905165195465
loss 200 0.16062724590301514
loss 300 0.1566394716501236
loss 400 0.16018106043338776
2
loss 0 0.15196411311626434
loss 100 0.1418580412864685
loss 200 0.13676388561725616
loss 300 0.13600683212280273
loss 400 0.13815969228744507
3
loss 0 0.13282470405101776
loss 100 0.12614336609840393
loss 200 0.1215393990278244
loss 300 0.1223907619714737
loss 400 0.12370844185352325
4
loss 0 0.1198616772890091
loss 100 0.11553604900836945
loss 200 0.11100683361291885
loss 300 0.11266766488552094
loss 400 0.11343987286090851
5
loss 0 0.11060134321451187
loss 100 0.10780564695596695
loss 200 0.10320667177438736
loss 300 0.10539567470550537
loss 400 0.10586313158273697
6
loss 0 0.10361592471599579
loss 100 0.10182967036962509
loss 200 0.09717686474323273
loss 300 0.09967273473739624
loss 400 0.100010856